In [ ]:
%load_ext tensorboard

from headers import *
from common import CreateAndSaveModel
from common import GetTrainTestSets
from common import ClearTrainingOutput
 
def BuildModel(x_learn_layers = 311, learn_rate = 0.0001, l1 = None, l2 = None, top_layer_cnt = 1, units = 512, version = 0):
    basemodel = BaseModel(learn_rate = learn_rate, L1 = l1, L2 = l2,
                          top_layer_cnt = top_layer_cnt, units = units, version = version)
    tunner_model = InceptionV3(include_top=False, weights='imagenet')
    return PredefinedModel(model = tunner_model, top_layers = x_learn_layers, intercept_module = basemodel).Create()

# 3 vs 4

In [ ]:
train_set, test_set = GetTrainTestSets([[3],[4]], [[2048], [2048]], [[1024], [1024]])
keras_tuning_project_name = 'inception_v_three'
tunning_epoch_cnt = 5

## (3 vs 4): Keras Tuning 

In [ ]:
#put the weight information into the memory for each configuration and epoch
#so this method could consume tens of gigabays of memory
#devide one configuration into little ones 
def model_builder(hp): 
    hp_units = hp.Choice('units', values = [512,1024])
    hp_top_layer_cnt = hp.Choice('top_layer_cnt', values = [1,2])
    hp_l1 = hp.Choice('l1', values = [0.0, 0.0001, 0.0005, 0.001])
    hp_l2 = hp.Choice('l2', values = [0.0, 0.0001, 0.0005, 0.001])
    hp_learn_rate = hp.Choice('learn_rate', values = [0.001, 0.0001])
    predefinedModel = BuildModel(learn_rate = hp_learn_rate, l1 = hp_l1, l2 = hp_l2,
                          top_layer_cnt = hp_top_layer_cnt, units = hp_units)
    return predefinedModel.model

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = tunning_epoch_cnt,
                     factor = 2,
                     directory = 'keras_tuning',
                     project_name = keras_tuning_project_name) 

In [ ]:
tuner.search(train_set, epochs = tunning_epoch_cnt, validation_data = test_set, callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
tuner.results_summary()
#mmodels = tuner.get_best_models(num_models = 3)

## (3 vs 4): Tuning1

In [ ]:
EPOCH = 10
predefinedModel = BuildModel(learn_rate = 0.0001, l1 = 0.0001, l2 = 0.0001,top_layer_cnt = 1, units = 512)
extra_callbacks = [TensorBoard(log_dir="logs", histogram_freq=1)]

In [ ]:
predefinedModel.fit(train_set = train_set, val_set = test_set, epochs = EPOCH,
                log = True, threshold = 0.7,extra_callbacks = extra_callbacks)

### (3 vs 4): Save Model1

In [ ]:
CreateAndSaveModel(root_dir = '', model = predefinedModel.model, test_set,
                   name = predefinedModel.getDescription(), extent = 'hdf5')